# Stochastic processes and reproducibility

## Generating random numbers

In [1]:
import agentpy as ap
import random

To illustrate, let us define a model that generates a list of ten pseudo-random numbers:

In [2]:
class RandomModel(ap.Model):
    
    def setup(self):
        self.random_numbers = [random.randint(0, 9) for _ in range(10)]
        print(f"Model {self.p.n} generated the numbers {self.random_numbers}")

Now if we run this model multiple times, we will get a different series of numbers:

In [3]:
for i in range(2):
    parameters = {'steps':0, 'n':i}
    model = RandomModel(parameters)
    results = model.run(display=False)

Model 0 generated the numbers [9, 3, 3, 8, 8, 0, 1, 9, 4, 7]
Model 1 generated the numbers [0, 5, 9, 4, 6, 5, 3, 2, 2, 0]


In [4]:
for i in range(2):
    parameters = {'seed':1, 'steps':0, 'n':i}
    model = RandomModel(parameters)
    model.run(display=False)

Model 0 generated the numbers [2, 9, 1, 4, 1, 7, 7, 7, 6, 3]
Model 1 generated the numbers [2, 9, 1, 4, 1, 7, 7, 7, 6, 3]


## Using multiple generators

In [5]:
class RandomAgent2(ap.Agent):
    
    def setup(self):
        seed = model.seed_generator.getrandbits(128)  # Get seed from model
        self.random = random.Random(seed)  # Create generator for this agent
        self.random_numbers = [self.random.randint(0, 9) for _ in range(10)]
        print(f"{self} generated the numbers {self.random_numbers}")
        
class RandomModel2(ap.Model):
    
    def setup(self):
        self.seed_generator = random.Random(self.p.model_seed)
        self.add_agents(2, RandomAgent2)

for i in range(2):
    print(f"Model {i}:")
    parameters = {'model_seed': 1, 'steps': 0}
    model = RandomModel2(parameters)
    results = model.run(display=False)
    print()

Model 0:
RandomAgent2 (Obj 1) generated the numbers [8, 7, 0, 1, 2, 3, 9, 4, 5, 0]
RandomAgent2 (Obj 2) generated the numbers [8, 1, 4, 6, 6, 3, 4, 3, 5, 1]

Model 1:
RandomAgent2 (Obj 1) generated the numbers [8, 7, 0, 1, 2, 3, 9, 4, 5, 0]
RandomAgent2 (Obj 2) generated the numbers [8, 1, 4, 6, 6, 3, 4, 3, 5, 1]



Alternatively, we could also have each agent start from the same seed:

In [6]:
class RandomAgent3(ap.Agent):
    
    def setup(self):
        self.random = random.Random(self.p.agent_seed)
        self.random_numbers = [self.random.randint(0, 9) for _ in range(10)]
        print(f"{self} generated the numbers {self.random_numbers}")
        
class RandomModel3(ap.Model):
    
    def setup(self):
        self.add_agents(2, RandomAgent3)
        
for i in range(2):
    print(f"\nModel {i}:")
    parameters = {'agent_seed': 1, 'steps':0, 'n':i}
    model = RandomModel3(parameters)
    results = model.run(display=False)


Model 0:
RandomAgent3 (Obj 1) generated the numbers [2, 9, 1, 4, 1, 7, 7, 7, 6, 3]
RandomAgent3 (Obj 2) generated the numbers [2, 9, 1, 4, 1, 7, 7, 7, 6, 3]

Model 1:
RandomAgent3 (Obj 1) generated the numbers [2, 9, 1, 4, 1, 7, 7, 7, 6, 3]
RandomAgent3 (Obj 2) generated the numbers [2, 9, 1, 4, 1, 7, 7, 7, 6, 3]


## Modeling stochastic processes

This section presents some stochastic operations that are often used in agent-based models. To start, we prepare a generic model with ten agents:

In [7]:
model = ap.Model()
agents = model.add_agents(10)
agents

AgentList [10 agents]

If we look at the agent's ids, we see that they have been created in order:

In [8]:
agents.id

AttrList of attribute 'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [9]:
agents.shuffle().id

AttrList of attribute 'id': [5, 10, 3, 9, 6, 4, 7, 1, 8, 2]

In [10]:
agents.random(5).id

AttrList of attribute 'id': [6, 9, 10, 3, 5]

In [11]:
for _ in range(2):
    custom_generator = random.Random(1)
    print(agents.random(5, custom_generator).id)

AttrList of attribute 'id': [3, 10, 6, 5, 9]
AttrList of attribute 'id': [3, 10, 6, 5, 9]


In [12]:
choices = random.choices(agents, k=5, weights=agents.id)

In [13]:
ap.AgentList(choices).id

AttrList of attribute 'id': [5, 4, 5, 8, 7]

## Further reading

- Random number generation in Python:  
  https://realpython.com/python-random/
- Stochasticity in agent-based models:  
  http://www2.econ.iastate.edu/tesfatsi/ace.htm#Stochasticity
- Pseudo-random number generators:  
  https://en.wikipedia.org/wiki/Pseudorandom_number_generator
- What is random:  
  https://www.youtube.com/watch?v=9rIy0xY99a0